In [2]:
import numpy as np
import time

## Generate data

In [3]:
# In real world, you cannot learn how the data was generated. So do not rely on this function when coding your lab.
def generate_data(dim, num):
    x = np.random.normal(0, 10, [num, dim])
    coef = np.random.uniform(-1, 1, [dim, 1])
    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0
    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
    return x, label, mislabel/num

## Model class

In [4]:
class SVM1: # with soft margin
    def __init__(self,C,maxIter):
        """
        Parameters:
        C: a constant
        maxIter: the maximum iteration number 
        """
        self.C = C
        self.maxIter = maxIter
       
    def args(self,x,y):
        self.b = 0.0
        self.m = np.shape(x)[0]
        self.n = np.shape(x)[1]
        self.x = x
        self.y = y

        self.alpha = np.zeros(self.m)
        self.eCache = [self.calculate_ei(i) for i in range(self.m)] # save all the ei in a list
        self.err = [abs(y[i] * self.fxi(i) - 1) for i in range(self.m)] # the degree that betray KKT

    def KKT(self,i): # satisfy KKT
        temp = self.fxi(i) * self.y[i]

        if self.alpha[i] == 0:
            return temp >= 1
        elif 0 < self.alpha[i] < self.C:
            return temp == 1
        elif self.alpha[i] == self.C:
            return temp <= 1
    
    def fxi(self,i):
        r = 0
        for j in range(self.m):
            r += self.alpha[j] * self.y[j] * np.dot(self.x[i],self.x[j])
        return r + self.b

    def calculate_ei(self,i):
        """
        Calculate error: ei = fxi - yi
        """
        return self.fxi(i) - self.y[i]

    def clip_alpha(self,aj,H,L):
        if aj > H:
            aj = H
        if aj < L:
            aj = L
        return aj

    def initalpha(self):
        temp = 0.0
        for i in range(self.m):
            if self.KKT(i): # satisfy KKT
                continue
            else:
                erri = self.err[i]
                if erri > temp:
                    temp = erri
                    new_i = i
                else:
                    continue

        ei = self.eCache[new_i]
        if ei >= 0: # find the maximum gap between i and j
            j = min(range(self.m),key = lambda x: self.eCache[x])
        else:
            j = max(range(self.m),key = lambda x: self.eCache[x])
        return new_i,j

    def fit(self,X,y):
        self.args(X,y)

        for t in range(self.maxIter):
            i,j = self.initalpha()

            if(self.y[i] != self.y[j]):
                zeta = self.alpha[i] - self.alpha[j]
                L = max(0,-zeta) # lower bounds
                H = min(self.C,self.C - zeta) # upper bounds
            else:
                zeta = self.alpha[i] + self.alpha[j]
                L = max(0,zeta - self.C)
                H = min(zeta,self.C)

            ei = self.eCache[i]
            ej = self.eCache[j]

            eta = np.dot(self.x[i],self.x[i]) + np.dot(self.x[j],self.x[j]) - 2 * np.dot(self.x[i],self.x[j]) 
            if eta <= 0:
                continue
            alphaj_new = self.alpha[j] + self.y[j] * (ei - ej)/eta
            alphaj_new = self.clip_alpha(alphaj_new,H,L)
            alphai_new = self.alpha[i] + self.y[i] * self.y[j] * (self.alpha[j] - alphaj_new)

            # renew the  
            self.alpha[i] = alphai_new
            self.alpha[j] = alphaj_new

            self.eCache[i] = self.calculate_ei(i)
            self.eCache[j] = self.calculate_ei(j)
            self.err[i] = abs(y[i] * self.fxi(i) - 1)
            self.err[j] = abs(y[j] * self.fxi(j) - 1)

            # renew b
            support = []
            for i in range(self.alpha.shape[0]):
                if 0 < self.alpha[i] < self.C:
                    support.append(i)
            num = len(support)
            ans = 0
            for s in range(num):
                temp = 0
                for j in range(num):
                    temp += self.alpha[j] * y[j] * np.dot(self.x[j],self.x[s])
                ans += (1/self.y[s] - temp)
            
            self.b = ans / num 
        
        return 'train done!'

    def predict(self, X):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        """
        ans = 0
        for i in range(len(X)):
            ans += self.alpha[i] * self.y[i] * np.dot(self.x[i],X)
        ans += self.b
        if ans > 0:
            return 1
        else:
            return -1

    def accuracy(self,X,y):
        right_count = 0
        for i in range(X.shape[0]):
            result = self.predict(X[i])
            if result == y[i]:
                right_count += 1
        return right_count / X.shape[0]

In [5]:
class SVM2: # gradient decreasing method
    def __init__(self, dim, maxIter, learning_rate, W):
        """
        Parameters:
        dim: the number of attributes
        maxIter: the maximum iteration number
        learning_rate: learning_rate
        W: the weight
        """
        self.dim = dim
        self.maxIter = maxIter
        self.learning_rate = learning_rate
        self.W = W

    def fit(self, X, y):
        """
        Fit the coefficients via gradient decrease methods
        X: train set
        y: the label of train set
        """
        X = X.T
        temp = np.zeros(X.shape[1])
        # add a row which is all 1
        X = np.row_stack((X,temp))
        d = X.shape[0] - 1

        grad = np.zeros(self.W.shape)
        for iter in range(self.maxIter):
            for i in range(d):
                ans = 0
                for n in range(len(y)):
                    if y[n] * np.dot(self.W.T,X[:,n]) < 1:
                        ans += (-y[n] * X[i][n])
                    else:
                        ans += 0
                grad[i] = ans
                self.W[i] -= self.learning_rate * grad[i]


    def predict(self, X):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        """
        X = X.T
        temp = np.zeros(X.shape[1])
        # add a row which is all 1
        X = np.row_stack((X,temp))

        pred = []
        for i in range(X.shape[1]):
            pred.append(np.dot(self.W.T,X[:,i]))

        for i in range(X.shape[1]):
            if pred[i] > 0:
                pred[i] = 1
            else:
                pred[i] = -1
        return pred

    def accuracy(self, X, y):
        right_count = 0
        ans = self.predict(X)
        for i in range(len(y)):
            if ans[i] == y[i]:
                right_count += 1
        return right_count / len(X)
    
    def watch(self,X):
        print(self.predict(X))

## Construct and train the models

In [6]:
# generate data
m = 10 # dimension of sample
n = 1000 # number of sample
X_data,y_data,mislabel = generate_data(m,n)
print("The mislabel is",mislabel)

The mislabel is 0.029


In [7]:
# split data by choosing 75% as train set
k = int(0.75*n)
X_train = X_data[0:k]
y_train = y_data[0:k]
X_test = X_data[k:n]
y_test = y_data[k:n]

#### SMO

In [ ]:
# construct model and train
start = time.time()
model1 = SVM1(C = 1.0,maxIter = 100)
model1.fit(X_train,y_train)
end = time.time()
print("SVM1's time is: {:.4f} seconds.".format(end - start))

#### Gradient decrease

In [8]:
# construct model and train
dim1 = X_train.shape[1] # the number of attributes
W = np.zeros((dim1 + 1,1))
start = time.time()
model2 = SVM2(dim1,1000,0.01,W)
model2.fit(X_train,y_train)
end = time.time()
print("SVM2's time is: {:.4f} seconds.".format(end - start))

SVM2's time is: 45.4348 seconds.


## Predict and compare the results

In [ ]:
# SVM1
print("SVM1's test accuracy: {:.4f}".format(model1.accuracy(X_test,y_test)))
# SVM2
print("SVM2's test accuracy:{:.4f}".format(model2.accuracy(X_test,y_test)))

In [ ]:
# using sklearn:
from sklearn.svm import SVC
start = time.time()
clf = SVC(kernel = "linear")    
clf.fit(X_train,y_train)
end = time.time()
accuracy = clf.score(X_test,y_test)
print("The test accuracy of SVM in sklearn is: {:.4f}".format(accuracy))
print("Sklearn time is: {:.4f} seconds.".format(end - start))